In [ ]:
# Install libs (run once)
!pip install -q joblib scikit-learn

In [ ]:
# AI-Based Fitness & Indian Diet Planner

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error,accuracy_score,f1_score,recall_score

In [ ]:
PEOPLE_CSV = "/content/final_engineered_dataset.csv"
FOODS_CSV = "/content/master_food_dataset_v5_clean.csv"

people = pd.read_csv(PEOPLE_CSV)
foods = pd.read_csv(FOODS_CSV)

# Ensure BMI exists
people["BMI"] = people["Weight_kg"] / (people["Height_cm"]/100)**2

def bmi_category(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif bmi < 25:
        return "Normal"
    elif bmi < 30:
        return "Overweight"
    elif bmi < 35:
        return "Obesity I"
    elif bmi < 40:
        return "Obesity II"
    else:
        return "Obesity III"

people["BMI_Category"] = people["BMI"].apply(bmi_category)

foods["name_lower"] = foods["Food_items"].str.lower()

people.head(), foods.head()


(   Age  Gender  Height_cm  Weight_kg      Goal Vegetarian Diabetic High_BP  \
 0   30    Male      170.0       70.0  maintain         No       No      No   
 1   30  Female      170.0       70.0  maintain         No       No      No   
 2   30    Male      170.0       70.0  maintain         No       No      No   
 3   30  Female      170.0       70.0  maintain         No       No      No   
 4   30    Male      170.0       70.0  maintain         No       No      No   
 
   Activity_Level  Dietary_Restrictions  ...  target_carb_g  sodium_limit_mg  \
 0       Moderate                   NaN  ...          368.1             2300   
 1       Moderate                   NaN  ...          334.5             2300   
 2       Moderate                   NaN  ...          376.0             2300   
 3       Moderate                   NaN  ...          327.7             2300   
 4       Moderate                   NaN  ...          363.4             2300   
 
    cal_breakfast  cal_lunch  cal_dinner  

In [ ]:
required_cols = ["Age","Gender","Height_cm","Weight_kg","BMI",
                 "Goal","Vegetarian","Diabetic","High_BP","Activity_Level"]

for col in required_cols:
    if col not in people.columns:
        people[col] = None

people["Age"] = people["Age"].fillna(30).astype(int)
people["Height_cm"] = people["Height_cm"].fillna(170).astype(float)
people["Weight_kg"] = people["Weight_kg"].fillna(70).astype(float)


people.head()


,Age,Gender,Height_cm,Weight_kg,Goal,Vegetarian,Diabetic,High_BP,Activity_Level,Dietary_Restrictions,...,target_carb_g,sodium_limit_mg,cal_breakfast,cal_lunch,cal_dinner,cal_snacks,recommended_weekly_exercise_hours,exercise_type,Diet_Recommendation,BMI_Category
0,30,Male,170.0,70.0,maintain,No,No,No,Moderate,NaN,...,368.1,2300,551,776,876,301,3.28,Moderate (Cardio + Strength),Balanced,Normal
1,30,Female,170.0,70.0,maintain,No,No,No,Moderate,NaN,...,334.5,2300,509,717,809,277,2.05,Moderate (Cardio + Strength),Balanced,Normal
2,30,Male,170.0,70.0,maintain,No,No,No,Moderate,NaN,...,376.0,2300,561,790,892,306,2.55,Moderate (Cardio + Strength),Balanced,Normal
3,30,Female,170.0,70.0,maintain,No,No,No,Moderate,NaN,...,327.7,2300,500,704,795,273,2.45,Moderate (Cardio + Strength),Balanced,Normal
4,30,Male,170.0,70.0,maintain,No,No,No,Moderate,NaN,...,363.4,2300,545,768,867,297,3.47,Moderate (Cardio + Strength),Balanced,Normal


In [ ]:
if "Weekly_Exercise_Hours" not in people.columns:

    def generate_hours(row):
        bmi = row["BMI"]
        goal = str(row["Goal"]).lower()

        if goal == "lose":
            base = 4.0
        elif goal == "gain":
            base = 2.0
        else:
            base = 2.5

        if bmi > 30:
            base += 1.0
        elif bmi < 18:
            base -= 0.8

        return round(base + np.random.uniform(-0.5, 0.8), 2)

    people["Weekly_Exercise_Hours"] = people.apply(generate_hours, axis=1)

people["Weekly_Exercise_Hours"].head()


,Weekly_Exercise_Hours
0,2.29
1,2.32
2,2.93
3,2.63
4,3.12


In [ ]:
if "Daily_Caloric_Intake" not in people.columns:

    def compute_bmr(row):
        w = row["Weight_kg"]
        h = row["Height_cm"]
        a = row["Age"]
        if str(row["Gender"]).lower() == "male":
            return 10*w + 6.25*h - 5*a + 5
        else:
            return 10*w + 6.25*h - 5*a - 161

    people["BMR"] = people.apply(compute_bmr, axis=1)

    def act_mult(level):
        level = str(level).lower()
        if "low" in level: return 1.2
        if "moderate" in level: return 1.55
        if "high" in level: return 1.75
        return 1.4

    people["activity_factor"] = people["Activity_Level"].apply(act_mult)

    people["Daily_Caloric_Intake"] = (
        people["BMR"] * people["activity_factor"] *
        np.random.uniform(0.95, 1.05, len(people))
    ).astype(int)

people["Daily_Caloric_Intake"].head()


,Daily_Caloric_Intake
0,2535
1,2283
2,2430
3,2180
4,2619


In [ ]:
people["Gender_encoded"] = people["Gender"].map({"Male": 0, "Female": 1}).fillna(0).astype(int)


In [ ]:
X = people[["Age","Gender_encoded","Weight_kg","Height_cm","BMI"]]
y = people["BMI_Category"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model_bmi = RandomForestClassifier()
model_bmi.fit(X_train, y_train)

pred = model_bmi.predict(X_test)
print("BMI Category Accuracy:", accuracy_score(y_test, pred))


BMI Category Accuracy: 1.0


In [ ]:
y_ex = people["Weekly_Exercise_Hours"]
model_ex = RandomForestRegressor()
model_ex.fit(X, y_ex)

RandomForestRegressor()

In [ ]:
def calculate_calories(gender, age, weight, height, activity_level, goal):

    # BMR (Mifflin St Jeor)
    if gender == "Male":
        BMR = 10*weight + 6.25*height - 5*age + 5
    else:
        BMR = 10*weight + 6.25*height - 5*age - 161

    # Activity multipliers
    mult = {"Low": 1.35, "Moderate": 1.55, "High": 1.75}
    TDEE = BMR * mult.get(activity_level, 1.55)

    # Goal adjustment
    if goal == "lose":
        TDEE *= 0.80
    elif goal == "gain":
        TDEE *= 1.15

    return round(TDEE)

In [ ]:
def get_meal_pool(meal_type, veg, diabetic, high_bp):
    df = foods.copy()

    if meal_type == "Breakfast":
        df = df[df["meal_category"] == "Breakfast"]
    elif meal_type == "Lunch":
        df = df[df["meal_category"] == "Lunch/Dinner"]
    elif meal_type == "Dinner":
        df = df[df["meal_category"] == "Lunch/Dinner"]
    else:
        df = df[df["meal_category"] == "Snack"]

    if veg == "Yes":
        df = df[df["is_veg_final"] == True]

    if diabetic == "Yes":
        df = df[df["diabetic_safe"] == True]

    if high_bp == "Yes":
        df = df[df["bp_safe"] == True]

    return df.reset_index(drop=True)

In [ ]:
def choose_items(df, target_cal, max_items=2):
    items = []
    cur = 0
    for _, row in df.sample(frac=1).iterrows():
        if len(items) >= max_items:
            break
        if cur + row["calories"] <= target_cal:
            items.append(row)
            cur += row["calories"]
    return items

In [ ]:
def structured_meal(user, meal_type, cal_target):
    pool = get_meal_pool(meal_type, user["Vegetarian"], user["Diabetic"], user["High_BP"])
    pool = pool.sample(frac=1).reset_index(drop=True)

    selected = []

    # -------------------------
    # BREAKFAST
    # -------------------------
    if meal_type == "Breakfast":
        mains = pool[pool["calories"] >= 160]
        sides = pool[pool["calories"] < 160]
        if len(mains):
            selected += choose_items(mains, cal_target*0.8, max_items=2)
        if len(sides):
            selected += choose_items(sides, cal_target*0.2, max_items=1)

    # -------------------------
    # LUNCH (THALI)
    # -------------------------
    elif meal_type == "Lunch":
        carbs = pool[pool["Food_items"].str.contains("Chapati|Roti|Rice|Biryani", case=False)]
        proteins = pool[pool["protein_g"] >= 10]
        veggies = pool[pool["protein_g"] < 10]

        if len(carbs):
            selected += choose_items(carbs, cal_target*0.33, max_items=1)
        if len(proteins):
            selected += choose_items(proteins, cal_target*0.37, max_items=1)
        if len(veggies):
            selected += choose_items(veggies, cal_target*0.30, max_items=1)

    # -------------------------
    # DINNER (LIGHT MEAL)
    # -------------------------
    elif meal_type == "Dinner":
        light_carbs = pool[pool["Food_items"].str.contains("Chapati|Roti|Phulka", case=False)]
        proteins = pool[pool["protein_g"] >= 10]

        if len(light_carbs):
            selected += choose_items(light_carbs, cal_target*0.45, max_items=1)
        if len(proteins):
            selected += choose_items(proteins, cal_target*0.55, max_items=1)

    # -------------------------
    # SNACKS
    # -------------------------
    else:
        selected += choose_items(pool, cal_target, max_items=2)

    # -------------------------------------------------------------
    #  CALORIE BOOSTER — INSIDE FUNCTION (THIS FIXES YOUR ERROR)
    # -------------------------------------------------------------
    current_cal = sum(x["calories"] for x in selected)
    lower_limit = cal_target * 0.95
    upper_limit = cal_target * 1.10

    small_items = pool[pool["calories"] <= 200]

    while current_cal < lower_limit and len(small_items) > 0:
        extra = small_items.sample(1).iloc[0]
        if current_cal + extra["calories"] <= upper_limit:
            selected.append(extra)
            current_cal += extra["calories"]
        else:
            break

    return selected


In [ ]:
def create_full_day_plan(user, total_cal):

    cal_b = int(total_cal * 0.25)
    cal_l = int(total_cal * 0.35)
    cal_d = int(total_cal * 0.30)
    cal_s = total_cal - (cal_b + cal_l + cal_d)

    b = structured_meal(user, "Breakfast", cal_b)
    l = structured_meal(user, "Lunch", cal_l)
    d = structured_meal(user, "Dinner", cal_d)
    s = structured_meal(user, "Snack", cal_s)

    all_items = b + l + d + s

    totals = {
        "cal": int(sum(x["calories"] for x in all_items)),
        "protein": float(sum(x["protein_g"] for x in all_items)),
        "fat": float(sum(x["fat_g"] for x in all_items)),
        "carb": float(sum(x["carb_g"] for x in all_items)),
        "sugar": float(sum(x["sugar_g"] for x in all_items)),
        "sodium": int(sum(x["sodium_mg"] for x in all_items)),
    }

    def fmt(meal):
        clean = []
        for m in meal:
            name = m["Food_items"].split("#")[0].strip()
            clean.append(f"{name} ({int(m['calories'])} kcal)")
        return " ; ".join(clean)

    return {
        "breakfast_plan": fmt(b),
        "lunch_plan": fmt(l),
        "dinner_plan": fmt(d),
        "snacks_plan": fmt(s),
        "totals": totals
    }


In [ ]:
def predict_and_plan_final(age, gender, weight, height, goal,
                           vegetarian, diabetic, high_bp, activity_level):

    user_bmi = weight / (height/100)**2

    X_temp = pd.DataFrame([[
        age,
        0 if gender=="Male" else 1,
        weight,
        height,
        user_bmi
    ]], columns=["Age","Gender_encoded","Weight_kg","Height_cm","BMI"])

    bmi_pred = model_bmi.predict(X_temp)[0]
    ex_hours = model_ex.predict(X_temp)[0]

    calories = calculate_calories(gender, age, weight, height, activity_level, goal)

    user = {"Vegetarian": vegetarian, "Diabetic": diabetic, "High_BP": high_bp}

    mp = create_full_day_plan(user, calories)

    return {
        "BMI_Category": bmi_pred,
        "Predicted_Exercise_Hours_weekly": ex_hours,
        "Daily_Calories": calories,
        "Meal_Plan": mp
    }

In [ ]:
def get_user_input():
    print("=== AI Fitness & Diet Planner ===")
    age = int(input("Enter Age: "))
    gender = input("Gender (Male/Female): ").strip().title()
    weight = float(input("Weight (kg): "))
    height = float(input("Height (cm): "))
    goal = input("Goal (lose/gain/maintain): ").strip().lower()
    vegetarian = input("Vegetarian? (Yes/No): ").strip().title()
    diabetic = input("Diabetic? (Yes/No): ").strip().title()
    high_bp = input("High BP? (Yes/No): ").strip().title()
    activity = input("Activity Level (Low/Moderate/High): ").strip().title()
    return age, gender, weight, height, goal, vegetarian, diabetic, high_bp, activity

def run_planner():
    age, g, w, h, goal, v, d, bp, act = get_user_input()
    result = predict_and_plan_final(age, g, w, h, goal, v, d, bp, act)

    print("\n=== RESULTS ===")
    print(f"BMI Category: {result['BMI_Category']}")
    print(f"Exercise Hours/week: {result['Predicted_Exercise_Hours_weekly']:.2f}")
    print(f"Daily Calories Needed: {result['Daily_Calories']}")

    print("\n=== MEAL PLAN ===")
    df = pd.DataFrame({
        "Meal":["Breakfast","Lunch","Dinner","Snacks",
                "Total Calories","Protein (g)","Fat (g)","Carbs (g)",
                "Sugar (g)","Sodium (mg)"],
        "Plan":[
            result["Meal_Plan"]["breakfast_plan"],
            result["Meal_Plan"]["lunch_plan"],
            result["Meal_Plan"]["dinner_plan"],
            result["Meal_Plan"]["snacks_plan"],
            result["Meal_Plan"]["totals"]["cal"],
            result["Meal_Plan"]["totals"]["protein"],
            result["Meal_Plan"]["totals"]["fat"],
            result["Meal_Plan"]["totals"]["carb"],
            result["Meal_Plan"]["totals"]["sugar"],
            result["Meal_Plan"]["totals"]["sodium"],
        ]
    })

    display(df)

In [ ]:
run_planner()

=== AI Fitness & Diet Planner ===
Enter Age: 34
Gender (Male/Female): mslr
Weight (kg): 79
Height (cm): 203
Goal (lose/gain/maintain): lose
Vegetarian? (Yes/No): no
Diabetic? (Yes/No): yes
High BP? (Yes/No): yes
Activity Level (Low/Moderate/High): high

=== RESULTS ===
BMI Category: Normal
Exercise Hours/week: 2.66
Daily Calories Needed: 2419

=== MEAL PLAN ===


,Meal,Plan
0,Breakfast,Masala Dosa (240 kcal) ; Uttapam (210 kcal) ; ...
1,Lunch,Chapati (111 kcal) ; Fish Curry (278 kcal) ; R...
2,Dinner,Chapati (136 kcal) ; Fish Curry (278 kcal) ; A...
3,Snacks,Dhokla (175 kcal) ; Coffee (8 kcal)
4,Total Calories,2235
5,Protein (g),88.009785
6,Fat (g),74.696191
7,Carbs (g),242.065821
8,Sugar (g),21.256433
9,Sodium (mg),2858
